# SNOTEL Data Access

This notebook allows for easy access to snow depths and SWE from the Snow Telemetry (SNOTEL) network. A simple example is used to show quick access to SNOTEL data over Alexander Lake, AK using the `easysnowdata` package.

Credit: Eric Gagliano (egagli@uw.edu) for the `easysnowdata` package, which in turn uses `ulmo` to access SNOTEL data. Note that development of `ulmo` has stalled, so SNOTEL sites developed after 2021 are not accessible through this method.

In [ ]:
import easysnowdata as esd

We start by calling all of the available stations through `easysnowdata`. In doing so, we have access to several variables for each station:
* `code`: ID for the station of interest.
* `name`: Name of the station site.
* `network`: Name of the weather station network ("CCSS" or "SNOTEL")
* `elevation_m`: Elevation of the station site, in meters.
* `latitude` and `longitude`: Latitude/longitude of site.
* `state`: State of SNOTEL site.
* `HUC`: Hydrologic Unit Code for the site.
* `mgrs`: Military Grid Reference System coordinate of site.
* `mountainRange`: Mountain range of site, if any.
* `beginDate` and `endDate`: Begin and ending data of available station data.

In [ ]:
# Call available weather stations, including SNOTEL
stations = esd.automatic_weather_stations.StationCollection()

In [ ]:
stations.all_stations

For this example, we will look at Alexander Lake in Alaska. Based on the table above, Alexander Lake has the `code` **1267_AK_SNTL**.

In [ ]:
# Subset data to Alexander Lake in Alaska
alex_lake = esd.automatic_weather_stations.StationCollection()
alex_lake.get_data('1267_AK_SNTL')

In [ ]:
alex_lake.data

A summary of the loaded data:
* `TAVG`, `TMIN`, `TMAX`: Average/minimum/maximum daily temperature, in Celsius.
* `SNWD`: Daily snow depth, in meters.
* `WTEQ`: Daily snow water equivalent, in meters.
* `PRCPSA`: Daily snow accumulation, in meters.

Now that we have the data loaded, let's check out the location of the SNOTEL site.

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt

# Plot a map showing the location of the SNOTEL site, with an ESRI basemap
f, ax = plt.subplots()

alex_lake.stations.plot(ax=ax, color="red")

ax.set_xlim(-150.92, -150.86)
ax.set_ylim(61.74, 61.76)

ctx.add_basemap(
    ax, crs=alex_lake.stations.crs, source=ctx.providers.Esri.WorldImagery
)

We can also check out a time series of the snow depth and SWE during the 2018 water year.

In [ ]:
import pandas as pd

f, ax = plt.subplots(figsize=(12, 5))

# Line plots of snow depth and snow water equivalent
alex_lake.data["SNWD"].plot(ax=ax, label="snow depth")
alex_lake.data["WTEQ"].plot(ax=ax, label="snow water equivalent")

ax.set_xlim(pd.to_datetime(["2017-10-01", "2018-09-30"]))

ax.grid()
ax.legend()

ax.set_xlabel("time")
ax.set_ylabel("snow depth / SWE [meters]")
ax.set_title("Snow depth and SWE at Alexander Lake, AK \n(water year 2018)")

f.tight_layout()

The `easysnowdata` package also has useful utilities for determining the day of water year and water year. Using these, we can easily calculate and plot snow depth statistics for a water year.

In [ ]:
# Calculate day of water year (DOWY) and water year (WY)
alex_lake.data["DOWY"] = alex_lake.data.index.map(
    esd.utils.datetime_to_DOWY
)
alex_lake.data["WY"] = alex_lake.data.index.map(
    esd.utils.datetime_to_WY
)

In [ ]:
# Compute snow depth statistics
stat_list = ["min", "max", "mean", "std", "median"]
alex_lake_DOWY_snwd_stats = alex_lake.data.groupby("DOWY").agg(stat_list)[
    "SNWD"
]
alex_lake_DOWY_snwd_stats

In [ ]:
import datetime

# Determine water year of present day
today = datetime.datetime.today().strftime("%Y-%m-%d")
current_WY = slice(f"{int(today[0:4])-1}-10-01", f"{today}")
current_WY_al_snotel = alex_lake.data[current_WY.start : current_WY.stop]

In [ ]:
f, ax = plt.subplots(figsize=(12, 7))

# Plot snow depth statistics of each day in water year
for stat, stat_color in zip(
    ["min", "max", "mean", "median"], ["red", "blue", "mediumpurple", "mediumseagreen"]
):
    ax.plot(
        alex_lake_DOWY_snwd_stats.index,
        alex_lake_DOWY_snwd_stats[stat],
        label=stat,
        color=stat_color,
        linewidth=3,
    )

# Plot uncertainty shading
ax.fill_between(
    alex_lake_DOWY_snwd_stats.index,
    alex_lake_DOWY_snwd_stats["mean"] - alex_lake_DOWY_snwd_stats["std"],
    alex_lake_DOWY_snwd_stats["mean"] + alex_lake_DOWY_snwd_stats["std"],
    color="mediumpurple",
    alpha=0.3,
    label="mean +/- 1 std",
)

# Scatter plot of depths from current water year
ax.scatter(
    current_WY_paradise_snotel.DOWY,
    current_WY_paradise_snotel.SNWD,
    marker="o",
    color="black",
    label="Current WY",
)

ax.set_xlim([0, 366])
ax.set_ylim([0, 1.5])

ax.grid()
ax.legend()

ax.set_title(
    f"Current snow depth against historical snow depth stats by DOWY at Alexander Lake, AK\n({alex_lake.data.index.min().date()} - {alex_lake.data.index.max().date()})"
)
ax.set_xlabel("Day of Water Year [Oct 1 - Sept 30]")
ax.set_ylabel("Snow depth [meters]")
f.tight_layout()